
# Update local field via Alma API

update several bib records through the alma api.
Here, we add a local field 990 with a subfield $u containing an url to a thumbnail picture.

To run this script, add the following:

Write your Alma API key in a file named config.py in the same folder as your script. The file should look like this:


    api_key_nz = "NZ_API_KEY"

(This file is under .gitignore, the API keys are not publicly shared)


In [6]:
import requests
import urllib.request
import json
import xml.etree.ElementTree as ET
import re
import config

def check_datafield_exact_match(marcxml, tag, code, value):
    """
    Checks if a marc field with an exact match exists and returns a boolean value.
    
    Parameters:
        marcxml: MARCXML element
        tag (str): The tag of the MARC field.
        code (str): the code of the subfield
        value (str): The value of the MARC field.
    
    Returns:
        True or False
    """
    
    field = marcxml.findall(".//datafield[@tag='{}']/subfield[@code='{}']".format(tag, code))
    fieldmatch = False
    for f in field:
        if f.text == value:
            #debug: print("Field match")
            fieldmatch = True
    return fieldmatch

def check_datafield_string_match(marcxml, tag, code, value):
    """
    Checks if a marc field matches a certain string value and returns a boolean value.
    
    Parameters:
        marcxml: MARCXML element
        tag (str): The tag of the MARC field.
        code (str): the code of the subfield
        value (str): The value that should be matched in the MARC subfield
    
    Returns:
        True or False
    """
    
    field = marcxml.findall(".//datafield[@tag='{}']/subfield[@code='{}']".format(tag, code))
    fieldmatch = False
    for f in field:
        if value in f.text:
            #debug: 
            print("Fieldmatch in field:", f.text)
            fieldmatch = True
    return fieldmatch

def create_marc_field(tag, indicators=None, subfields=None):
    """
    Creates a new MARC field with the specified tag, value, indicators, and subfields.
    
    Parameters:
        tag (str): The tag of the MARC field.
        indicators (list, optional): The indicators of the MARC field. Defaults to None.
        subfields (dict, optional): The subfields of the MARC field. Defaults to None.
    
    Returns:
        str: The formatted MARC field as a string.
    """
        
    if indicators is not None:
        if isinstance(indicators, list):
            ind1 = f"{indicators[0]}"
            ind2 = f"{indicators[1]}"
            
        else:
            raise ValueError("Indicators must be a list of two elements.")
    else:
        ind1 = " "
        ind2 = " "
    
    field = ET.Element("datafield", attrib={"ind1": ind1, "ind2": ind2, "tag": tag})
    
    if subfields is not None:
        if isinstance(subfields, dict):
            for code, data in subfields.items():
                subfield = ET.Element("subfield", attrib={"code": code})
                subfield.text = data
                field.append(subfield)
        else:
            raise ValueError("Subfields must be a dictionary.")
    else:
        raise ValueError("Subfields should not be empty")
    print("New field:\n", ET.tostring(field))
    return field


# Set your Alma API key, base API endpoint, input file name
api_key = config.api_key_rzs
base_url = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1'
# file with all records:
#filename = 'mmsId_ThumnailUrl.json'
# testfile with 1 record, mmsid: 9910909220105505:
#filename = 'test-1-thumbnail.json'
# testfile with 2 records, one KORPLUZ / 9910964970105505, one ZHBFREE / 9913488850105505
filename = 'test-2-thumbnail.json'

provenanceZ = 'ZHBFREE'
provenanceK = 'KORPLUZ'


# Opening JSON file and return JSON object as a dictionary
f = open(filename)
id_thumbnail_list = json.load(f)
# debug: print(id_url_list)

# Loop through the list and update each bib record
for bib in id_thumbnail_list:
    mmsId = bib['mmsId']
    #debug: 
    print(mmsId)
    thumbnailUrl = bib['thumbnailUrl']
    # Construct the API URL to get the bib record in MARCXML format
    bib_url = f'{base_url}/bibs/{mmsId}?view=full&expand=None&apikey={api_key}'
    #debug:     print(bib_url)

    # Send the request to get the bib record
    response = requests.get(bib_url)
    print("Request response:", response.status_code)

    # Parse the MARCXML data from the response content
    marcxml = ET.fromstring(response.content)  
    #print("marcxml before update:", marcxml)
    
    # check if 990u field / subfield exists already:
    f990u_exists = check_datafield_exact_match(marcxml,'990','u',thumbnailUrl)
    print("Field 990 u exists:", f990u_exists)
    
    # Create a new 990 field with subfields u (url) and 3 (description)
    if f990u_exists == False:
        record = marcxml.find("record")
        new_990 = create_marc_field('990', indicators=None, subfields={"u": thumbnailUrl, "3": "Thumbnail", "9": "LOCAL"})
        record.append(new_990)
        
    # check if provenance field exists and change subfield i:
    f990i_z = check_datafield_string_match(marcxml,'990','i', provenanceZ)
    print("Field 990 i = ZHBFREE:", f990i_z)
    f990i_k = check_datafield_string_match(marcxml,'990','i', provenanceK)
    print("Field 990 i = KORPLUZ:", f990i_k)
    
    if (f990i_z or f990i_k) == True:
        # change subfield i :
        for field in marcxml.findall('.//datafield[@tag="990"]'):
            # Find the subfield with provenance
            provenance_subfield = field.find('.//subfield[@code="i"]')
            if provenance_subfield is not None:
            # Replace the field with the new value
                if f990i_z == True:
                    provenance_subfield.text = provenanceZ
                elif f990i_k == True:
                    provenance_subfield.text = provenanceK
                else: #debug
                    print("Error in provenance")
                
                # debug: 
                print("New subfield i value:", provenance_subfield.text)
               
    # before updating the record, check for order of MARC tags:   
    #record = marcxml.find("record")
    record[:] = sorted(record, key=lambda field_or_contr: field_or_contr.get('tag', '000'))

    # Convert the updated MARCXML back to a string
    updated_marcxml = ET.tostring(marcxml, encoding='utf-8')
    #debug:       
    print("Updated XML: \n", updated_marcxml)
    
    # Construct the API URL to update the bib record
    update_url = f'{base_url}/bibs/{mmsId}?apikey={api_key}'
    print("update request url: ", update_url)

    '''
    
    # Send the request to update the bib record with the updated MARCXML
    response = requests.put(update_url, data=updated_marcxml, headers={'Content-Type': 'application/xml'})
    #debug:
    print("Updated XML (response content): \n", response.content)

    # Print the response status code to verify the update was successful
    print(f'Bib record {mmsId} updated - Response code: {response.status_code}')
    #debug: break

    '''




9910964970105505
Request response: 200
Field 990 u exists: False
New field:
 b'<datafield ind1=" " ind2=" " tag="990"><subfield code="u">https://zentralgut.ch/content/9910964970105505/150/0/9910964970105505_BB.jpg</subfield><subfield code="3">Thumbnail</subfield><subfield code="9">LOCAL</subfield></datafield>'
Field 990 i = ZHBFREE: False
Fieldmatch in field: http://edoc.zhbluzern.ch/sosa/03/SYS-001096497_BB.jpg -- KORPLUZ
Field 990 i = KORPLUZ: True
New subfield i value: KORPLUZ
Updated XML: 
 b'<bib><mms_id>9910964970105505</mms_id><record_format>marc21</record_format><linked_record_id type="NZ">991073875149705501</linked_record_id><title>[Kapellbr\xc3\xbccke]</title><author>Lienert, Rudolf</author><network_numbers><network_number>(IDSLU)001096497ILU01</network_number><network_number>011942762</network_number><network_number>(swissbib)011942762-41slsp_network</network_number><network_number>(EXLNZ-41SLSP_NETWORK)991073875149705501</network_number></network_numbers><place_of_publicati